# Selección para Jimenez-Esteban

Primero se seleccionan en la lista de Jimenez-Estaban las estrellas que sólo están en grupos de 2 (binarias). Después, se pegan en la misma fila para el tratamiento posterior.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Cuales pares (o grupos comóviles) aparecen  en ambos catálogos

# Importar la tabla de J-E y organizar por GroupID
je_raw = pd.read_csv('../data_raw/comovingGaiaDR2_1557810818.csv').rename(columns={'#RA':'RA'}).sort_values(by='GroupID')

Seleccionamos sólo aquellas que están en grupos de 2

In [5]:
je_raw = je_raw[je_raw.GroupSize == 2]

Hacer el crossmatch (cuidando de no repetir datos comunes)

In [6]:
je_cross = pd.merge(je_raw,je_raw[['ra1','dec1','Source_GaiaDR2','GroupID','radial_velocity','radial_velocity_error','parallax','parallax_total_error',
                                   'pmra_corrected','pmra_corrected_error','pmdec_corrected','pmdec_corrected_error',
                                   'phot_g_mean_mag','phot_bp_mean_mag', 'phot_rp_mean_mag', 'RUWE', 'HRV', 'e_HRV',
                                   'D_G', 'Teff', 'Teff_error', 'Av', 'Mass', 'Mass_error', 'td','td_error', 'tms', 'tms_error', 'ECAC']],
                    how='outer',on='GroupID')

Las unimos para que aparezcan en un solo renglon

In [7]:
je_cross = je_cross[(je_cross.Sep_ang > 0.) &
                    (je_cross.Source_GaiaDR2_x != je_cross.Source_GaiaDR2_y)]

Omitimos las columnas que no van a ser útiles

In [9]:
je_cross = je_cross.drop(columns=['RA','DEC','dis','Source_Tycho2','sbname','sbname_link','GroupID','GroupSize'])

Hacemos la selección de calidad de J-E y salvamos:

- Con GDR2, encuentran 3741 candidatos a binarias abiertas (pair AND multiple systems)
- Usan la mag límite de Ty2 = 11.5 para M completo a 40 pc
                                   G completo a 200 pc
                                   F completo a 400 pc
- Con errores < 10% en p.m. y parallax
- Con separaciones < 500,000 AU
- Igual paralaje y p.m. en r.a. y dec (dentro de 2.5$\sigma$ => 0.13 mas; 0.2 mas/yr)
- Sin corrección geométrica.
    Esto da 3852 candidatos (dobles + múltiples), 11 834 estrellas
- Gaia DR2 + RAVE da vr para 3318 de las estrellas.
- Con errores < 5 km/s, se reduce a 2877 de Gaia DR-2 y 282 de RAVE
- Sólo para 678 pares había RV para ambas componentes.
- 16% (108) fueron discrepantes en RV, 66 de los múltiples también
- En total, 570 pares con RV concordantes

In [16]:
#je_cross[je_cross.Sep_sky > 495000]
je_cross = je_cross[ True &
# Quitamos las que no sean concordantes         
          (np.abs((je_cross.radial_velocity_y/je_cross.radial_velocity_x-1.)/
                  (je_cross.radial_velocity_error_x/je_cross.radial_velocity_x +
                   je_cross.radial_velocity_error_y/je_cross.radial_velocity_y)) < 2.5 ) &
#         (np.abs(je_cross.pmra_corrected_y-je_cross.pmra_corrected_x) < 2.5*je_cross.pmra_corrected_error_y) &
#         (np.abs(je_cross.pmra_corrected_y-je_cross.pmra_corrected_x) < 2.5*je_cross.pmra_corrected_error_x) &
#         (np.abs(je_cross.pmdec_corrected_y-je_cross.pmdec_corrected_x) < 2.5*je_cross.pmdec_corrected_error_y) &
#         (np.abs(je_cross.pmdec_corrected_y-je_cross.pmdec_corrected_x) < 2.5*je_cross.pmdec_corrected_error_x) &
# Quitamos las que no tienen velocidad radial en GDR2         
         (~(je_cross.radial_velocity_x.isna() | je_cross.radial_velocity_y.isna())) &
# Quitamos las que tengan errores mayores a 5km/s         
         (je_cross.radial_velocity_error_x < 5) &
         (je_cross.radial_velocity_error_y < 5) &
#         (~(je_cross.HRV_x.isna() | je_cross.HRV_y.isna())) &
#         (je_cross.e_HRV_x < 5) &
#         (je_cross.e_HRV_y < 5) &
         True ]

Salvamos la lista:

In [17]:
je_cross.to_pickle('../data_processed/je_seleccion_best.pkl')